In [ ]:
"the purpose of this file is to take a yolo file labels and convert them to coco format to be used by DETR"

In [1]:
import torch

In [3]:
ds = "data/Dataset"
# ds = f"military drones images"

models = f"/mnt/d/23b6034/FYP/models/DETR-4"
train_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/train"
test_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/test"
val_dataset = f"/mnt/d/23b6034/FYP/datasets/{ds}/valid"

In [4]:
class DroneDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, label_dir, classes, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.classes = classes
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + '.txt') # Assuming label files have the same name as images with a .txt extension

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    class_id, center_x, center_y, width, height = map(float, line.strip().split())
                    # Convert YOLO format (center_x, center_y, width, height) to [x_min, y_min, x_max, y_max]
                    # Assuming coordinates are normalized (0 to 1)
                    x_min = center_x - width / 2
                    y_min = center_y - height / 2
                    x_max = center_x + width / 2
                    y_max = center_y + height / 2
                    boxes.append([x_min, y_min, x_max, y_max])
                    labels.append(int(class_id)) # Assuming class_id is an integer

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)

        return image, target

In [5]:
# Define image and label directory paths
labels = "/labels"
train_label_dir = train_dataset + labels
val_label_dir = val_dataset + labels
test_label_dir = test_dataset + labels

In [6]:
import os
# Print a sample label file from the training set
sample_label_file = None
for f in os.listdir(train_label_dir):
    if f.endswith('.txt'):
        sample_label_file = os.path.join(train_label_dir, f)
        break
if sample_label_file:
    print(f"Sample label file: {sample_label_file}")
    with open(sample_label_file, 'r') as file:
        print(file.read())
else:
    print("No label files found in the training label directory.")

Sample label file: /mnt/d/23b6034/FYP/datasets/data/Dataset/train/labels/-B8-DA-C1-F8-2778_jpg.rf.45ca35bb4ad15328a4decfe25bbb208c.txt
3 0.814990234375 0.26957413249211354 0.047939453125 0.21794952681388013
3 0.755947265625 0.19501577287066246 0.063037109375 0.24089905362776023
3 0.699130859375 0.29610410094637224 0.0452734375 0.11328075709779178
3 0.591708984375 0.2358832807570978 0.055927734375 0.20504731861198738
3 0.499375 0.18569400630914829 0.039951171875 0.11900630914826499



In [9]:
from PIL import Image
import os

# Function to convert YOLO to COCO format
# YOLO: class_id, center_x, center_y, width, height (normalized)
# COCO: class_id, x_min, y_min, width, height (absolute)
def convert_yolo_to_coco(label_path, image_path):
    coco_labels = []
    with open(label_path, 'r') as f:
        lines = f.readlines()
    img = Image.open(image_path)
    img_w, img_h = img.size
    for line in lines:
        class_id, center_x, center_y, w, h = map(float, line.strip().split())
        x_min = (center_x - w / 2) * img_w
        y_min = (center_y - h / 2) * img_h
        width = w * img_w
        height = h * img_h
        coco_labels.append(f"{int(class_id)} {x_min:.6f} {y_min:.6f} {width:.6f} {height:.6f}")
    return coco_labels

# Only change labels, keep images untouched
for split, base_dir, label_dir in [
    ("train", train_dataset, train_label_dir),
    ("val", val_dataset, val_label_dir),
    ("test", test_dataset, test_label_dir)
]:
    img_dir = os.path.join(base_dir, "images")
    coco_label_dir = os.path.join(os.path.dirname(label_dir), "labels-coc", split)
    os.makedirs(coco_label_dir, exist_ok=True)
    if not os.path.exists(img_dir):
        print(f"Image directory not found: {img_dir}")
        continue
    for img_file in os.listdir(img_dir):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(label_dir, label_file)
            coco_label_path = os.path.join(coco_label_dir, label_file)
            if os.path.exists(label_path):
                img_path = os.path.join(img_dir, img_file)
                coco_labels = convert_yolo_to_coco(label_path, img_path)
                with open(coco_label_path, 'w') as f:
                    for line in coco_labels:
                        f.write(line + '\n')
            else:
                # If no label, create an empty file
                open(coco_label_path, 'w').close()
print("YOLO labels converted to COCO format and saved in labels-coc folders. Images untouched.")

YOLO labels converted to COCO format and saved in labels-coc folders. Images untouched.


In [8]:
# Debug: List image and label files for test split
print("Test image directory:", test_dataset)
print("Test label directory:", test_label_dir)

image_files = [f for f in os.listdir(test_dataset) if f.endswith(('.jpg', '.jpeg', '.png'))]
label_files = [f for f in os.listdir(test_label_dir) if f.endswith('.txt')]

print("Image files:", image_files)
print("Label files:", label_files)

if image_files:
    first_img = image_files[0]
    expected_label = os.path.splitext(first_img)[0] + '.txt'
    print(f"For image '{first_img}', expected label file: '{expected_label}'")
    print("Exists in label dir:", expected_label in label_files)
else:
    print("No image files found in test directory.")

Test image directory: /mnt/d/23b6034/FYP/datasets/data/Dataset/test
Test label directory: /mnt/d/23b6034/FYP/datasets/data/Dataset/test/labels
Image files: []
Label files: ['490_png_jpg.rf.c37a7b498dd0a60feeb0a4a612dfd3e6.txt', '490_png_jpg.rf.fe0acdfa0437f050e0cc3295b51d3823.txt', '491_png_jpg.rf.175fc24594a1e78077124b222c38fae0.txt', '491_png_jpg.rf.b65af9860e4a5c12bf99adc07769915d.txt', '491_png_jpg.rf.b7156f6fb74265dba24920a1a711f7c7.txt', '491_png_jpg.rf.e9c894e1f892a87168cd44e6c82e56c4.txt', '492_png_jpg.rf.01f43533631d061fa8008e2f1f1823fd.txt', '492_png_jpg.rf.3bdd9d62ccdfb25a52dadc0366ebbf7e.txt', '492_png_jpg.rf.436d02fbddbd6018b0683f3dd472437a.txt', '492_png_jpg.rf.6ed01463eabc182a42efdbd77134f08f.txt', '492_png_jpg.rf.b0d3b38291358698459448167f1a07ad.txt', '492_png_jpg.rf.d89710155ff64d07bc75938585dd1c22.txt', '493_png_jpg.rf.2f487ae65524aa3007dff280c5bd8d97.txt', '493_png_jpg.rf.7537bc6e976d01c7d56733674c398ed8.txt', '494_png_jpg.rf.1710e7527f31146ca99877b48f8fda15.txt', '4